In [1]:
import os 
import json
import pandas as pd
import numpy as np
import scanpy as sc
from tqdm import tqdm
from glob import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [45]:
mahmood_files = glob('/home/than/.cache/huggingface/hub/datasets--MahmoodLab--hest-bench/snapshots/78ed54aad495c764f8f48c1f53204cd4f9e7743d/*/var_50genes.json')
len(mahmood_files)
tasks = []
genes = []
for file in mahmood_files:
    with open(file) as f:
        data = json.load(f)
    gene = data['genes']
    task = file.split('/')[-2]
    genes.append(gene)
    tasks.append(task)

gene_dict = dict(zip(tasks, genes))
with open ('high_var_genes.json', 'w') as f:
    json.dump(gene_dict, f, indent=4)

10

In [3]:
df = pd.read_csv("hf://datasets/MahmoodLab/hest/HEST_v1_0_2.csv")
# df.head()
meta_df = df[df['id'].isin(['TENX95','TENX99','NCBI783','NCBI785'])]

In [35]:
adata_list = []
for _, row in tqdm(meta_df.iterrows(), total=meta_df.shape[0]):
    id = row['id']
    adata = sc.read_h5ad(f"/home/than/Datasets/HEST_data/st/{id}.h5ad")
    adata.var_names_make_unique()
    adata_list.append(adata)

100%|██████████| 4/4 [00:00<00:00, 42.29it/s]


In [38]:
df_cancer = pd.read_csv('/home/than/DeepLearning/PORPOISE/datasets_csv/signatures.csv')
cancer_genes = []
for col in df_cancer.columns:
    cancer_genes.extend(df_cancer[col].dropna().tolist())
cancer_genes = list(set(cancer_genes))

common_genes = None
stacked_expressions = None
min_cells_pct = 0.1
k = 50

# Get the common genes
for adata in tqdm(adata_list):
    my_adata = adata.copy()
    if min_cells_pct:
        sc.pp.filter_genes(my_adata, min_cells=np.ceil(min_cells_pct * len(my_adata.obs)))
    curr_genes = np.array(my_adata.to_df().columns)
    if common_genes is None:
        common_genes = curr_genes
    else:
        common_genes = np.intersect1d(common_genes, curr_genes)
common_genes = [gene for gene in common_genes if 'BLANK' not in gene and 'Control' not in gene]
for adata in tqdm(adata_list):
    if stacked_expressions is None:
        stacked_expressions = adata.to_df()[common_genes]
    else:
        stacked_expressions = pd.concat([stacked_expressions, adata.to_df()[common_genes]])
stacked_adata = sc.AnnData(stacked_expressions.astype(np.float32))
sc.pp.filter_genes(stacked_adata, min_cells=0)
sc.pp.log1p(stacked_adata)
sc.pp.highly_variable_genes(stacked_adata, n_top_genes=k)
top_k = stacked_adata.var_names[stacked_adata.var['highly_variable']][:k].tolist()

100%|██████████| 4/4 [00:00<00:00, 237.73it/s]
/home/than/anaconda3/envs/hest/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/than/anaconda3/envs/hest/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/than/anaconda3/envs/hest/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [39]:
len(top_k)
mahmood_genes = ["ABCC11", "ADH1B", "ADIPOQ", "ANKRD30A", "AQP1", "AQP3", "CCR7", "CD3E", "CEACAM6", "CEACAM8", "CLIC6", "CYTIP", "DST", "ERBB2", "ESR1", "FASN", "GATA3", "IL2RG", "IL7R", "KIT", "KLF5", "KRT14", "KRT5", "KRT6B", "MMP1", "MMP12", "MS4A1", "MUC6", "MYBPC1", "MYH11", "MYLK", "OPRPN", "OXTR", "PIGR", "PTGDS", "PTN", "PTPRC", "SCD", "SCGB2A1", "SERHL2", "SERPINA3", "SFRP1", "SLAMF7", "TACSTD2", "TCL1A", "TENT5C", "TOP2A", "TPSAB1", "TRAC", "VWF"]
len(mahmood_genes)
len(np.intersect1d(top_k, mahmood_genes))
# top_k = mahmood_genes
len(np.intersect1d(top_k, cancer_genes))

50

50

45

17

In [4]:
# breast_df = df[df['organ'] == 'Breast']
top_k = [
    "CXCR2P1", "ICOS", "TIMD4", "CTLA4", "PAX5", "KLRC3", "FCRL3", "AIM2", "GBP5", "CCL4"
]
breast_df = df
adata_list = []
ids = []
for _, row in tqdm(breast_df.iterrows(), total=breast_df.shape[0]):
    id = row['id']
    adata = sc.read_h5ad(f"/home/than/Datasets/HEST_data/st/{id}.h5ad")
    adata.var_names_make_unique()
    adata_list.append(adata)
    ids.append(id)
# Get the common genes
overlaps = []
spots = []
for adata in tqdm(adata_list):
    my_adata = adata.copy()
    if min_cells_pct:
        sc.pp.filter_genes(my_adata, min_cells=np.ceil(min_cells_pct * len(my_adata.obs)))
    curr_genes = np.array(my_adata.to_df().columns)
    common_genes = np.intersect1d(top_k, curr_genes)
    overlaps.append(len(common_genes))
    spots.append(len(my_adata.obs))

df = pd.DataFrame({'id': ids, 'overlap': overlaps, 'spots': spots})

  1%|          | 6/1108 [00:00<00:22, 49.64it/s]/home/than/anaconda3/envs/hest/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
  1%|          | 12/1108 [00:00<00:33, 33.04it/s]/home/than/anaconda3/envs/hest/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/than/anaconda3/envs/hest/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
 16%|█▌        | 175/1108 [00:22<01:29, 10.41it/s]

In [32]:
df = df.sort_values('overlap', ascending=False)
df = df[~ df['id'].isin(['TENX95','TENX99','NCBI783','NCBI785'])]
df = df[df['overlap'] > 25]
df['spots'].sum()
len(df[df['overlap'] > 25])

263818

84

In [7]:
df = df.sort_values('overlap', ascending=False).reset_index(drop=True)
df = df[df['overlap'] > 50]
df = df[~ df['id'].isin(['TENX95','TENX99','NCBI783','NCBI785'])]
df['spots'].sum()
df.head()
# df.head(60)

0

,id,overlap,spots


In [134]:
import scanpy.external as sce

id = 'NCBI776'
adata = sc.read_h5ad(f"/home/than/Datasets/HEST_data/st/{id}.h5ad")
# sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
print(adata.X.shape)
adata_new = sce.pp.magic(adata, name_list=top_k, knn=5)
print(adata_new.X.shape)

(4992, 18085)


/home/than/.local/lib/python3.9/site-packages/magic/magic.py:425: UserWarning: Input matrix contains unexpressed genes. Please remove them prior to running MAGIC.
  warnings.warn(


(4992, 98)


/home/than/.local/lib/python3.9/site-packages/magic/magic.py:484: UserWarning: genes ['ANGPT2' 'TPSAB1'] missing from input data
  warnings.warn(


In [129]:
curr_genes = np.array(adata_new.to_df().columns)
len(np.intersect1d(top_k, curr_genes))

90

In [22]:
len(common_genes)

379

In [ ]:
for adata in tqdm(adata_list):

    if stacked_expressions is None:
        stacked_expressions = adata.to_df()[common_genes]
    else:
        stacked_expressions = pd.concat([stacked_expressions, adata.to_df()[common_genes]])

In [2]:
df = pd.read_csv('stacked_expressions.csv')
df.head()

,Unnamed: 0
0,000x028
1,000x029
2,000x030
3,000x031
4,000x032


In [6]:
meta_df = pd.read_csv("hf://datasets/MahmoodLab/hest/HEST_v1_0_2.csv")

In [4]:
genes = get_k_genes_from_df(meta_df, 1000, criteria='var')

,dataset_title,id,image_filename,organ,disease_state,oncotree_code,species,patient,st_technology,data_publication_date,...,nb_genes,treatment_comment,pixel_size_um_embedded,pixel_size_um_estimated,magnification,fullres_px_width,fullres_px_height,tissue,disease_comment,subseries
0,FFPE Human Ovarian Cancer Data with Human Immu...,TENX142,TENX142.tif,Ovary,Cancer,HGSOC,Homo sapiens,NaN,Xenium,5/15/24,...,541,NaN,0.273769,0.273706,40x,21806,37330,Ovary,Ovary Serous Carcinoma; Stage II-A; Grade 3,NaN
1,FFPE Human Lung Cancer Data with Human Immuno-...,TENX141,TENX141.tif,Lung,Cancer,LUAD,Homo sapiens,patient 1,Xenium,5/15/24,...,541,NaN,0.273772,0.273643,40x,15513,39361,Lung,Lung NSCLC; Stage I-B; Grade 2,NaN
2,FFPE Human Pancreatic Ductal Adenocarcinoma Da...,TENX140,TENX140.tif,Pancreas,Cancer,PAAD,Homo sapiens,patient 3,Xenium,5/15/24,...,541,NaN,0.273766,0.273768,40x,19068,41014,Pancreas,Pancreatic Ductal Adenocarcinoma; Stage IIB; G...,NaN
3,FFPE Human Colorectal Cancer Data with Human I...,TENX139,TENX139.tif,Bowel,Cancer,COADREAD,Homo sapiens,NaN,Xenium,5/15/24,...,541,NaN,0.273773,0.273741,40x,21507,40815,Bowel,Colorectal Cancer Adenocarcinoma; Grade 2,NaN
4,FFPE Human Brain Cancer Data with Human Immuno...,TENX138,TENX138.tif,Brain,Cancer,GBM,Homo sapiens,NaN,Xenium,4/15/24,...,541,NaN,0.273775,0.273665,40x,38957,58584,Brain,NaN,NaN
